This is the code of add translation symmetry in Kitaev model. 

\section{Kitaev Hamiltonian}

\begin{equation}
    H_K = - \sum_{<j,k>_\alpha} J_\alpha S_j^\alpha S_k^\alpha
\end{equation}
where $\alpha = x, y, z$.

\section{Hamiltonian matrix}

\quad The Hamiltonian can be rewriten to $H_K = H_x + H_y +H_z$, where $H_z$ is the diagonal term and $H_x, H_y$ are off-diagonal terms:
\begin{align}
      H_x &= -\sum_{<i,j>_x} J_x S_i^x S_j^x = -\sum_{<i,j>_x} \frac{J_x}{4} ( S_i^+ S_j^- + S_i^- S_j^+ + S_i^+ S_j^+            + S_i^- S_j^-)\\
      H_y &= -\sum_{<i,j>_y} J_y S_i^y S_j^y = -\sum_{<i,j>_y} \frac{J_y}{4} ( S_i^+ S_j^- + S_i^- S_j^+ - S_i^+ S_j^+            - S_i^- S_j^-)\\
      H_z &= -\sum_{<i,j>_z} J_z S_i^z S_j^z
\end{align}
where $S_i^\pm = S_i^x \pm i S_i^y$.

\section{$ N_1 \times N_2 \times 2$ lattice}

\quad The lattice vectors are $\vec{n}_1 = (0,1), \vec{n}_2 = (\frac{1}{2},\frac{\sqrt{3}}{2})$. The number of unit cells along these two directions are $N_1$ and $N_2$ respectively. We label each unit cell by its row and colume numbers $(r, c)$(start from $0$), the label of this unit cell given by $ n = r \times N_1 + c$. We can label A atom in the $n$-th unit cell as $2n$ and B atom as $2n + 1$.

\quad For reference, the ground state energies of Kitaev model with different lattice sizes are (PBC):

| N1 | N2 | GS energy |
|---|---|---|
|2  | 3 | -0.204172555  |
|2  | 4 | -0.2022542475 |
|3  | 3 | -0.19849309   |
|3  | 4 | -0.198872775  |
|2  | 6 | -0.2003512    |


\section{Translational symmetry and Momentum basis}

\subsection{Translation operators}
\begin{align}
T_1^{N_1} = 1  &\Rightarrow k_1 = m_1 \frac{2 \pi}{N_1},\ m_1 = -N_1 //2 + 1 ,\dots, N_1//2 \\
T_2^{N_2} = 1  &\Rightarrow k_2 = m_2 \frac{2 \pi}{N_1},\ m_2 = -N_2 //2 + 1 ,\dots, N_2//2
\end{align}

\subsection{Monentum basis}

A momentum state can be constructed using a reference state $|a \rangle$ (a single state in z-component basis) and all its translations:
\begin{equation}
|a_{k_1,k_2} \rangle = \frac{1}{\sqrt{N_{1a}N_{2a}}} \sum_{r_1 = 0}^{N_1-1} \sum_{r_2 = 0}^{N_2-1}
                           e^{-ik_1 r_1 - ik_2 r_2} T_1^{r_1} T_2^{r_2} |a \rangle
\end{equation}

We can easily prove that:
\begin{equation}
T_j |a_{k_1,k_2} \rangle = e^{-ik_j} |a_{k_1,k_2} \rangle, \ j = 1,2
\end{equation}

To construct a complete set of normalizable orthogonal state for a given $\vec{k} = (k_1, k_2)$, the corresponding representatives must obey:

\begin{equation}
T_1^{r_1} T_2^{r_2} |a \rangle \neq |b \rangle, r_{1,2} = 0,\dots,N_{1,2}-1 
\end{equation}

If the state satisfy:
\begin{align}
T_1^{R_{1a}} |a \rangle &= |a \rangle\\
T_2^{R_{2a}} |a \rangle &= |a \rangle
\end{align}

We have:
\begin{equation}
N_{ja} = \frac{N_j^2}{R_{ja}},\ j=1,2
\end{equation}

We can only use one of the states that related by translation operations as the reference state, normally we choose the one with smallest tag. Therefore, the dimension of the Hilbert space is approximately reduced by $N_1N_2$ times, $D \approx 2^{N_1N_2} / (N_1N_2)$.

In [14]:
import numpy as np
from tkinter import _flatten
import itertools

from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh

from numba import jit

def StateConfig(tag, SiteNum):
    '''The configuration of the state with tag.
       Inputs: tag: tag of a state
               SiteNum: total site number   
       Output: b: binary form of tag. type: list'''
    b = bin(tag)[2:]
    b = b.rjust(SiteNum,'0') 
    return b

def flip(tag,i,j, SiteNum):
    '''Flip the spin on i,j site.
       Inputs:tag: tag of a state
              i,j : position of spins that are flipped
              SiteNum: total site number
       Output:ntag: The tag of new state, type: int '''
    f = pow(2, SiteNum - i -1) + pow(2, SiteNum - j -1)
    ntag = tag^f
    return ntag

def index(N1, N2, c, r, atom = 'A'):
    '''Calculate the index of the atoms in the (r,c) unit cell'''
    c = c % N1
    r = r % N2
    
    n = r * N1 + c
    if atom == 'A':
        ind = 2 * n
    elif atom == 'B':
        ind = 2 * n + 1
    else:
        return print('ERROR: The atom index should be A or B.')
    return ind

def cyclebits(n1,n2, tag, N1, N2):
    '''Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state'''
    dim = N1 * N2
    SiteNum = N1 * N2 * 2
    config = StateConfig(tag, SiteNum) 
    sub = list(config[::2] + config[1::2])
    a = np.array(sub).reshape(2,N2,N1)

    # cycle of columns
    Nsub = []
    if n1 != 0:
        for n in range(2):
            b = a[n,:,:].tolist()
            for i in range(N2):
                b[i] = b[i][n1:] + b[i][:n1]
            Nsub += list(_flatten(b))

    #cycle of rows
    if n2 != 0:
        for n in range(2):
            at = a[n,:,:].T 
            b = at.tolist()
            for i in range(N1):
                b[i] = b[i][n2:] + b[i][:n2]
            na = np.array(b).T      
            nb = na.tolist()
            Nsub += list(_flatten(nb))
    
    # Nsub = [A,A,A,B,B,B], Nconfig = [A,B,A,B,A,B]
    Nconfig = list(itertools.chain.from_iterable(zip(Nsub[:dim],Nsub[dim:])))
    Nconfig_str = "".join(Nconfig)
    #print(Nconfig_str)
    
    #Nconfig_str = "".join(Nsub)
    #print(Nconfig_str)
    ntag = int(Nconfig_str, 2)    
    return ntag

def MomBasis(k1, k2, N1, N2):
    '''Momentum basis list of a given k1, k2
       Inputs: k1: momentum in n1-direction
               k2: momentum in n2-direction
               N1: number of columns
               N2: number of rows
       Outpus: mbasis: momentum basis list
               Ra: list of corresponding cycle period 
                   R1: n1-direction 
                   R2: n2-direction'''
    SiteNum = N1 * N2 * 2
    dim = pow(2, SiteNum)
    
    basis = list(range(dim))
    m1 = []
    m2 = []
    
    R0 = [] 
    R1 = []
    R2 = []
  
    while basis !=[]: # Translation symmetry along n1-direction
        tag = basis[0]
        stag = basis[0]
        for n1 in range(1, N1 + 1):
            ntag = cyclebits(n1, 0, tag, N1, N2)
            if ntag < tag:
                basis.remove(ntag)
                if ntag < stag:
                    stag = ntag
            elif ntag > tag:
                basis.remove(ntag)
            elif ntag == tag:
                if k1%(N1/n1) == 0:
                    R0.append(n1)
                    m1.append(stag)
                basis.remove(ntag)
                break

    basis = m1[:]
    while basis !=[]: # Translation symmetry along n2-direction
        tag = basis[0]
        stag = basis[0]
        for n2 in range(1, N2 + 1):
            ntag = cyclebits(0, n2, tag, N1, N2)
            if ntag in basis:
                if ntag < tag:
                    basis.remove(ntag)
                    if ntag < stag:
                        stag = ntag
                elif ntag > tag:
                    basis.remove(ntag)
                elif ntag == tag:
                    if k2%(N2/n2) == 0:
                        R2.append(n2)
                        m2.append(stag)
                        ind = m1.index(stag)
                        R1.append(R0[ind])
                    basis.remove(ntag)
                    break  
            else:
                continue
    return m2, R1, R2


In [15]:
'''Hamiltonian generation'''
def Findstate(tag, mbasis, N1, N2):
    '''Find the reference state of an arbitrary state.
       Inputs: tag :tag of the input state
               mbasis: list of momentum basis
               N1, N2: number of columns and rows
       Outputs: ntag: tag of the corresponding reference state
                l : translation steps
                ntag and l will return negative values if tag is not in mbasis.'''
    
    SiteNum = N1 * N2 * 2
    
    l = - np.ones(2,int)
    count = 0
    b = StateConfig(SiteNum , tag)
    
    for n1 in range(1, N1 + 1):
        ntag = cyclebits(n1, 0, tag, N1, N2)
        for n2 in range(1, N2 + 1):
            ntag1 = cyclebits(0, n2, ntag, N1, N2)
            if ntag1 in mbasis:
                l[0] = n1 % N1
                l[1] = n2 % N2
                break
            else:
                pass
            count += 1
        if ntag1 in mbasis:
            break
        else:
            pass
        
    if count >= SiteNum/2:
        ntag1 = -1
    return ntag1, l 

def KitaevRhom_T(J, N1, N2, k1, k2):
    '''Kitaev Hamiltonian on N1 * N2 * 2 lattice for a given k (with translation symmetry)
       N1: number of unit cells in n1-direction, column number
       N2: number of unit cells in n1-direction, row number
    
       Inputes: 
           N1, N2: width and length of the rhomboid cluster
           k1, k2: momentund
        
       Outputs: The Non-zero Hamiltonian element H[i] is in the 
                position (row[i], col[i])
                H, row, col : type: list
        '''
    SiteNum = N1 * N2 * 2
    mb, R1, R2 = MomBasis(k1, k2, N1, N2)
    R = list(map(lambda x,y:x*y, R1,R2))

    dim = len(mb)
    
    fac = np.zeros(2, complex)
    fac[0] = -1j * 2 * np.pi / N1
    fac[1] = -1j * 2 * np.pi / N2
    
    numNZero = 0 # count the number of non-zero element
    data = []
    row = []
    col = []
    
    for tag in mb:
        si = StateConfig(tag, SiteNum) 
        ind  = mb.index(tag)
        temp = {} # A temporay dict
        
        for r in range(N2):
            for c in range(N1):
                ini = index(N1, N2, c, r, atom = 'A')
                
                '''Sx Sx'''
                next = index(N1, N2, c, r, atom = 'B')
                
                ntag = flip(tag, ini, next, SiteNum)
                rtag, l = Findstate(ntag, mb, N1, N2)
                if rtag > 0:
                    nind = mb.index(rtag)
                    if nind in temp:
                        temp[nind] += (-J[0] /4 * np.exp(fac[0]*k1*l[0]) * np.exp(fac[1]*k2*l[1]) *
                        np.sqrt(R[ind]/R[nind]))
                    else:
                        temp[nind] = (-J[0] /4 * np.exp(fac[0]*k1*l[0]) * np.exp(fac[1]*k2*l[1]) *
                        np.sqrt(R[ind]/R[nind]))
                                
                '''Sy Sy'''
                next = index(N1, N2, c - 1, r, atom = 'B')
                    
                ntag = flip(tag, ini, next, SiteNum)
                rtag, l = Findstate(ntag, mb, N1, N2)
                if rtag > 0:
                    nind = mb.index(rtag)
                    if nind in temp:
                        temp[nind] += (J[1] * (int(si[ini]) - 0.5) * (int(si[next]) - 0.5) *
                        np.exp(fac[0]*k1*l[0]) * np.exp(fac[1]*k2*l[1]) *
                        np.sqrt(R[ind]/R[nind]))
                    else:
                        temp[nind] = (J[1] * (int(si[ini]) - 0.5) * (int(si[next]) - 0.5) *
                        np.exp(fac[0]*k1*l[0]) * np.exp(fac[1]*k2*l[1]) *
                        np.sqrt(R[ind]/R[nind]))
                        

                '''Sz Sz'''
                next = index(N1, N2, c, r - 1, atom = 'B')
                    
                if ind in temp:
                    temp[ind] += - J[2] * (int(si[ini]) - 0.5) * (int(si[next]) - 0.5)
                else:
                    temp[ind] = - J[2] * (int(si[ini]) - 0.5)* (int(si[next]) - 0.5)
                        
        odTemp = sorted(temp)
        for sf in odTemp:
            col.append (ind)
            row.append (sf)
            data.append (temp[sf])
            numNZero += 1
            
    #print ('non zero elemens number = ', numNZero)
    #print('Percentage = ', numNZero/(dim * dim) * 100,'%')
    return col, row, data, dim


In [24]:
'''TEST CODE'''
N1 = 2
N2 = 3
#k1 = 0
#k2 = 0

SiteNum = N1 * N2 * 2


J = - np.array([1,1,1])
for k1 in range(N1):
    for k2 in range(N2):

        row = []
        col = []

        col, row, data, dim = KitaevRhom_T(J, N1, N2, k1, k2)

        ham = csc_matrix ((data, (row, col)), shape=(dim, dim))
        vals, vecs = eigsh (ham, k=5, which = 'SA')

        print (sorted(vals / SiteNum))

[-0.20417255625506717, -0.20411075388256703, -0.1952315716784578, -0.19473933720478007, -0.19364487526898488]
[-0.20417255625506645, -0.19523157167846236, -0.19364487526899088, -0.19364487526898844, -0.1903051875726507]
[-0.20417255625506628, -0.1952315716784608, -0.1936448752689889, -0.19364487526898685, -0.19030518757265147]
[-0.19364487526898955, -0.19364487526898577, -0.1935729712943617, -0.19030518757265136, -0.19030518757265094]
[-0.19364487526898952, -0.19364487526898808, -0.19030518757265424, -0.1903051875726505, -0.1870376099322675]
[-0.19364487526898785, -0.1936448752689878, -0.19030518757265327, -0.19030518757264914, -0.18703760993226493]


In [6]:
H1 = ham.T - ham

In [22]:
for i in range(dim):
    for j in range(dim):
        if H1[i,j] != 0:
            print('(',i, ',', j, ') = ', H1[i,j])

( 0 , 8 ) =  (-3.191891195797325e-16+0.4330127018922194j)
( 0 , 11 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 0 , 14 ) =  (-0.12499999999999979-0.2165063509461098j)
( 0 , 17 ) =  (0.12499999999999979+0.2165063509461098j)
( 0 , 116 ) =  (-0.1250000000000001+0.2165063509461096j)
( 0 , 117 ) =  (0.1250000000000001-0.2165063509461096j)
( 1 , 23 ) =  (-3.191891195797325e-16+0.4330127018922194j)
( 1 , 26 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 1 , 29 ) =  (-0.12499999999999979-0.2165063509461098j)
( 1 , 32 ) =  (0.12499999999999979+0.2165063509461098j)
( 1 , 332 ) =  (-0.1250000000000001+0.2165063509461096j)
( 1 , 333 ) =  (0.1250000000000001-0.2165063509461096j)
( 2 , 38 ) =  (-3.3306690738754696e-16+0.4330127018922194j)
( 2 , 41 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 2 , 44 ) =  (-0.12499999999999979-0.2165063509461098j)
( 2 , 47 ) =  (0.12499999999999978+0.2165063509461098j)
( 2 , 520 ) =  (-0.1250000000000001+0.2165063509461096j)
( 2 , 521 ) =  (0.125000000

( 22 , 957 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 23 , 1 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 23 , 349 ) =  (-3.191891195797325e-16+0.4330127018922194j)
( 23 , 352 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 23 , 355 ) =  (-0.12499999999999979-0.2165063509461098j)
( 23 , 358 ) =  (0.12499999999999979+0.2165063509461098j)
( 23 , 518 ) =  (0.1250000000000001-0.2165063509461096j)
( 23 , 523 ) =  (0.1250000000000001-0.2165063509461096j)
( 24 , 22 ) =  (0.25+0j)
( 24 , 27 ) =  (0.25+0j)
( 24 , 114 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 24 , 119 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 24 , 154 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 24 , 196 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 24 , 238 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 24 , 280 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 24 , 331 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 24 , 845 ) =  (3.7470027081099033e-16-0.43301

( 43 , 117 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 43 , 358 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 43 , 397 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 43 , 436 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 43 , 475 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 43 , 959 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 44 , 2 ) =  (0.12499999999999979+0.2165063509461098j)
( 44 , 40 ) =  (0.25+0j)
( 44 , 45 ) =  (0.25+0j)
( 44 , 50 ) =  (-0.25+0j)
( 44 , 750 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 44 , 780 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 44 , 810 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 45 , 44 ) =  (-0.25+0j)
( 45 , 47 ) =  (-0.25+0j)
( 45 , 521 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 45 , 610 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 45 , 613 ) =  (-0.12499999999999979-0.2165063509461098j)
( 45 , 616 ) =  (0.12499999999999979+0.2165063509461098j)
( 45 , 725 ) =  (-3.7470

( 73 , 75 ) =  (-0.25+0j)
( 73 , 85 ) =  (-0.25+0j)
( 73 , 118 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 73 , 330 ) =  (0.1250000000000001-0.2165063509461096j)
( 73 , 335 ) =  (0.1250000000000001-0.2165063509461096j)
( 73 , 353 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 73 , 392 ) =  (-3.191891195797325e-16+0.4330127018922194j)
( 73 , 431 ) =  (-0.1250000000000001+0.2165063509461096j)
( 73 , 470 ) =  (0.1250000000000001-0.2165063509461096j)
( 73 , 960 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 74 , 4 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 74 , 518 ) =  (0.1250000000000001-0.2165063509461096j)
( 74 , 523 ) =  (0.1250000000000001-0.2165063509461096j)
( 74 , 527 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 74 , 563 ) =  (-3.191891195797325e-16+0.4330127018922194j)
( 74 , 599 ) =  (-0.1250000000000001+0.2165063509461096j)
( 74 , 635 ) =  (0.1250000000000001-0.2165063509461096j)
( 75 , 73 ) =  (0.25+0j)
( 75 , 78 ) =  (0.25+0j)
( 75 , 114 ) =  

( 94 , 331 ) =  (0.1250000000000001-0.2165063509461096j)
( 94 , 334 ) =  (0.1250000000000001-0.2165063509461096j)
( 94 , 357 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 94 , 396 ) =  (-3.191891195797325e-16+0.4330127018922194j)
( 94 , 435 ) =  (-0.1250000000000001+0.2165063509461096j)
( 94 , 474 ) =  (0.1250000000000001-0.2165063509461096j)
( 94 , 961 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 95 , 5 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 95 , 519 ) =  (0.1250000000000001-0.2165063509461096j)
( 95 , 522 ) =  (0.1250000000000001-0.2165063509461096j)
( 95 , 531 ) =  (3.191891195797325e-16-0.4330127018922194j)
( 95 , 567 ) =  (-3.191891195797325e-16+0.4330127018922194j)
( 95 , 603 ) =  (-0.1250000000000001+0.2165063509461096j)
( 95 , 639 ) =  (0.1250000000000001-0.2165063509461096j)
( 96 , 94 ) =  (0.25+0j)
( 96 , 99 ) =  (0.25+0j)
( 96 , 115 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 96 , 118 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 96 , 

...

( 969 , 322 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 969 , 327 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 969 , 513 ) =  (-0.1250000000000001+0.2165063509461096j)
( 969 , 516 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 969 , 941 ) =  (-0.1250000000000001+0.2165063509461096j)
( 969 , 944 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 137 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 207 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 221 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 970 , 291 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 303 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 306 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 309 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 312 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 , 455 ) =  (-0.12499999999999979-0.2165063509461098j)
( 970 , 494 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 970 

( 982 , 347 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 982 , 412 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 982 , 425 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 982 , 457 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 982 , 458 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 982 , 463 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 982 , 490 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 982 , 1000 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 982 , 1016 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 982 , 1021 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 983 , 325 ) =  (-0.1250000000000001+0.2165063509461096j)
( 983 , 328 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 983 , 360 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 983 , 399 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 983 , 438 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 983 , 477 ) =  (3.7470027081099033e-16-0.4330127018922194j)
(

( 995 , 946 ) =  (-0.1250000000000001+0.2165063509461096j)
( 995 , 955 ) =  (-0.1250000000000001+0.2165063509461096j)
( 996 , 268 ) =  (-0.12499999999999979-0.2165063509461098j)
( 996 , 310 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 996 , 582 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 996 , 762 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 996 , 868 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 996 , 877 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 996 , 922 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 996 , 930 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 996 , 933 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 996 , 950 ) =  (-0.1250000000000001+0.2165063509461096j)
( 996 , 953 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 997 , 460 ) =  (-0.12499999999999979-0.2165063509461098j)
( 997 , 499 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 997 , 630 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 997 , 802 ) =  

( 1009 , 496 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1009 , 872 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1009 , 881 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1009 , 926 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1010 , 146 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 307 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 340 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 405 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 418 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1010 , 483 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 493 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 500 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 503 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 858 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1010 , 908 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1010 , 935 ) =  (3.7470027081099033e-16-0.

( 1023 , 465 ) =  (-0.12499999999999979-0.2165063509461098j)
( 1023 , 504 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1023 , 909 ) =  (-0.12499999999999979-0.2165063509461098j)
( 1023 , 936 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1024 , 273 ) =  (-0.12499999999999979-0.2165063509461098j)
( 1024 , 315 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1024 , 348 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1024 , 413 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1024 , 426 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1024 , 491 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1024 , 492 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1024 , 495 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1024 , 498 ) =  (3.7470027081099033e-16-0.4330127018922194j)
( 1024 , 501 ) =  (-3.7470027081099033e-16+0.4330127018922194j)
( 1024 , 1001 ) =  (-0.12499999999999979-0.2165063509461098j)
( 1024 , 1022 ) =  (3.7470027081099033e-16-0.4330127018

In [ ]:
N1 = 2
N2 = 1
k1 = 0
k2 = 0
tag = 0
mb, R1, R2 = MomBasis(k1, k2, N1, N2)
print(mb)
#ntag, l  = Findstate(tag, mb, N1, N2)
#print(ntag)
#print(l)

In [26]:
tag = 1
Findstate(tag, mb, N1, N2)

(1, array([0, 0]))